## Import Modules

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from os import listdir
from os.path import isfile, join
import time
import math
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt
import torchvision.transforms as T
#from torchsampler import ImbalancedDatasetSampler
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print('cuda available: '+ str(torch.cuda.is_available()))
#from skimage.morphology import disk, binary_dilation

from models_avm import NestedUNet, UNet
from loss_fun_avm import compute_per_channel_dice
from tra_val_avm import inference_out
from data_loader_avm import Dataset_out

cuda available: True


## Data Path

In [2]:
# Define data list here
# The files for the combination of detection and segmentation inference should be those slices with and without the bAVM.

path_tra1 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\1_tra\1' #mod
list_tra1 = os.listdir(path_tra1)
for i in range(len(list_tra1)):
    list_tra1[i] = path_tra1+'\\'+list_tra1[i]
path_tra2 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\1_tra\0' #mod
list_tra2 = os.listdir(path_tra2)
for i in range(len(list_tra2)):
    list_tra2[i] = path_tra2+'\\'+list_tra2[i]
    
list_tra = np.append(list_tra1,list_tra2)

path_val1 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\2_val\1' #mod
list_val1 = os.listdir(path_val1)
for i in range(len(list_val1)):
    list_val1[i] = path_val1+'\\'+list_val1[i]
path_val2 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\2_val\0' #mod
list_val2 = os.listdir(path_val2)
for i in range(len(list_val2)):
    list_val2[i] = path_val2+'\\'+list_val2[i]
    
list_val = np.append(list_val1,list_val2)

path_ts1 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\3_ts\1' #mod
list_ts1 = os.listdir(path_ts1)
for i in range(len(list_ts1)):
    list_ts1[i] = path_ts1+'\\'+list_ts1[i]
path_ts2 = r'E:\AVM_Project_TCVGH\MRI案\Datasets\prc\6_Datasets_for_model_fin2D\forUNetpp\3_ts\0' #mod
list_ts2 = os.listdir(path_ts2)
for i in range(len(list_ts2)):
    list_ts2[i] = path_ts2+'\\'+list_ts2[i]
    
list_ts = np.append(list_ts1,list_ts2)

## Output

In [3]:
# YOLOv8 & UNet++
# Define the detected label for the YOLO and then output the segmentation results

path_tra_lab_txt_root = [
    r'C:\Users\USER\Desktop\Detection\Results\yolov8\results_pt_aug_fin\1_tra\detect\labels' #mod
                   ]
path_val_lab_txt_root = [
    r'C:\Users\USER\Desktop\Detection\Results\yolov8\results_pt_aug_fin\2_val\detect\labels' #mod
                    ]
path_ts_lab_txt_root = [
    r'C:\Users\USER\Desktop\Detection\Results\yolov8\results_pt_aug_fin\3_ts\detect\labels' #mod
                    ]
mod_str = ['yolov8']

for m in range(1):
    
    path_tra_lab_txt = path_tra_lab_txt_root[m]
    path_val_lab_txt = path_val_lab_txt_root[m]
    path_ts_lab_txt = path_ts_lab_txt_root[m]

    # delete those did not detect by yolov5
    list_tra_lab_txt = os.listdir(path_tra_lab_txt)
    for i in range(len(list_tra_lab_txt)):
        list_tra_lab_txt[i] = os.path.splitext(list_tra_lab_txt[i])[0]    
    list_tra_fin = []
    for tmp in list_tra:
        for i in range(len(list_tra_lab_txt)):
            if list_tra_lab_txt[i] == os.path.split(tmp)[1][0:-4]:
                list_tra_fin.append(tmp)

    list_val_lab_txt = os.listdir(path_val_lab_txt)
    for i in range(len(list_val_lab_txt)):
        list_val_lab_txt[i] = os.path.splitext(list_val_lab_txt[i])[0]    
    list_val_fin = []
    for tmp in list_val:
        for i in range(len(list_val_lab_txt)):
            if list_val_lab_txt[i] == os.path.split(tmp)[1][0:-4]:
                list_val_fin.append(tmp)

    list_ts_lab_txt = os.listdir(path_ts_lab_txt)
    for i in range(len(list_ts_lab_txt)):
        list_ts_lab_txt[i] = os.path.splitext(list_ts_lab_txt[i])[0]    
    list_ts_fin = []
    for tmp in list_ts:
        for i in range(len(list_ts_lab_txt)):
            if list_ts_lab_txt[i] == os.path.split(tmp)[1][0:-4]:
                list_ts_fin.append(tmp)


    # load trained model
    PATH = [r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_nodia\train\exp2\best_val.tar', #mod
            r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_dia10\train\exp1\best_val.tar', #mod
            r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_dia20\train\exp1\best_val.tar', #mod
            r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_dia30\train\exp1\best_val.tar', #mod
            r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_dia40\train\exp1\best_val.tar', #mod
            r'C:\Users\USER\Desktop\Segmentation\Models\unetpp\Models\Seg_dia50\train\exp1\best_val.tar', #mod
           ]

    # score function
    score = [compute_per_channel_dice]

    # model dia
    mod_dia = ['nodia','dia10','dia20','dia30','dia40','dia50']
    # define the confidence threshold according to the highest f1 score
    conf_thresh = [0.240,0.256,0.161] # tra, val, ts #mod
    # define the path to save the results
    sav_path = r'C:\Users\USER\Desktop\Segmentation\Results\4_Results_unetpp_comb' #mod

    for i in range(len(mod_dia)):
        # Log and results saving directory
        path = sav_path+'\\'+mod_str[m]
        if os.path.isdir(path)==False:
            os.makedirs(path)
        path = sav_path+'\\'+mod_str[m]+'\\'+mod_dia[i]
        if os.path.isdir(path)==False:
            os.makedirs(path)

        # call model cuda for gpu
        model = NestedUNet(norm_method='group').cuda()
        own_state = torch.load(PATH[i])
        model.load_state_dict(own_state)
        model.eval();

        #i = 0
        for k in range(len(conf_thresh)):

            # Data loader
            tra_data = Dataset_out(list_tra_fin,
                                   path_tra_lab_txt,
                                   yolo_inf=True,
                                   conf_thresh=conf_thresh[k],
                                   dilate=True,
                                   dilate_factor=0)
            tra_loader_out = torch.utils.data.DataLoader(
                dataset = tra_data,           
                batch_size = 1,                 
                shuffle = False)

            val_data = Dataset_out(list_val_fin,
                                   path_val_lab_txt,
                                   yolo_inf=True,
                                   conf_thresh=conf_thresh[k],
                                   dilate=True,
                                   dilate_factor=0)
            val_loader_out = torch.utils.data.DataLoader(
                dataset = val_data,           
                batch_size = 1,                 
                shuffle = False)

            ts_data = Dataset_out(list_ts_fin,
                                  path_ts_lab_txt,
                                   yolo_inf=True,
                                   conf_thresh=conf_thresh[k],
                                   dilate=True,
                                   dilate_factor=0)
            ts_loader_out = torch.utils.data.DataLoader(
                dataset = ts_data,           
                batch_size = 1,                 
                shuffle = False)

            path_sav_tra = path + '\\1_tra'
            if os.path.isdir(path_sav_tra)==False:
                os.mkdir(path_sav_tra)
            path_sav_val = path + '\\2_val'
            if os.path.isdir(path_sav_val)==False:
                os.mkdir(path_sav_val)
            path_sav_ts = path + '\\3_ts'
            if os.path.isdir(path_sav_ts)==False:
                os.mkdir(path_sav_ts)

            inference_out(path_sav_tra,model,score,tra_loader_out)
            inference_out(path_sav_val,model,score,val_loader_out)
            inference_out(path_sav_ts,model,score,ts_loader_out)
            
            import gc
            gc.collect()
            torch.cuda.empty_cache()


|   100/ 2834 batches | ms/batch 8453.56 | 
|   200/ 2834 batches | ms/batch 7219.60 | 
|   300/ 2834 batches | ms/batch 6833.13 | 
|   400/ 2834 batches | ms/batch 6658.59 | 
|   500/ 2834 batches | ms/batch 7792.93 | 
|   600/ 2834 batches | ms/batch 8556.80 | 
|   700/ 2834 batches | ms/batch 8210.07 | 
|   800/ 2834 batches | ms/batch 5991.30 | 
|   900/ 2834 batches | ms/batch 7635.96 | 
|  1000/ 2834 batches | ms/batch 6690.53 | 
|  1100/ 2834 batches | ms/batch 7984.69 | 
|  1200/ 2834 batches | ms/batch 7252.14 | 
|  1300/ 2834 batches | ms/batch 8075.93 | 
|  1400/ 2834 batches | ms/batch 7716.30 | 
|  1500/ 2834 batches | ms/batch 5763.16 | 
|  1600/ 2834 batches | ms/batch 8883.43 | 
|  1700/ 2834 batches | ms/batch 7693.50 | 
|  1800/ 2834 batches | ms/batch 6897.95 | 
|  1900/ 2834 batches | ms/batch 7732.07 | 
|  2000/ 2834 batches | ms/batch 7970.31 | 
|  2100/ 2834 batches | ms/batch 7591.50 | 
|  2200/ 2834 batches | ms/batch 6087.60 | 
|  2300/ 2834 batches | ms/batch

# Log ^